In [4]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests 
from pandas.io.json import json_normalize 

In [5]:
!pip install folium
import pandas as pd
import folium

In [6]:
CLIENT_ID = 'LQJQLJ0WZYAHO00AXJJ5ZFBIN2CP4I3HT5YCUQDG2FXMW2DF' 
CLIENT_SECRET = 'HHKWKWAXGJ4WMMPC0Y5GMTJJPTEPPE45J1E5UQ021SKIGSE4' 
VERSION = '20200621' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: LQJQLJ0WZYAHO00AXJJ5ZFBIN2CP4I3HT5YCUQDG2FXMW2DF


In [19]:

LIMIT = 500 
cities = ["Istanbul", 'Izmir', 'Bursa', 'Ankara', 'Adana']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735") 
    results[city] = requests.get(url).json()

In [21]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [58]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

   
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=1,
            popup=label,
            color='yellow',
            fill=True,
            fill_color='#3286cc',
            fill_opacity=0.5,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of pizza places in Istanbul =  286
Showing Top 100
Total number of pizza places in Izmir =  185
Showing Top 100
Total number of pizza places in Bursa =  100
Showing Top 100
Total number of pizza places in Ankara =  201
Showing Top 100
Total number of pizza places in Adana =  114
Showing Top 100


In [61]:
maps[cities[0]]

In [60]:
maps[cities[1]]

In [62]:
maps[cities[2]]

In [64]:
maps[cities[3]]

In [65]:
maps[cities[4]]

In [47]:

maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='yellow',
            fill=True,
            fill_color='#3286cc',
            fill_opacity=0.5,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3286cc',
        fill_opacity=0.5,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Istanbul
Mean Distance from Mean coordinates
0.10142304550851947
Izmir
Mean Distance from Mean coordinates
0.04767834062530495
Bursa
Mean Distance from Mean coordinates
0.08822992634785252
Ankara
Mean Distance from Mean coordinates
0.12127488136768923
Adana
Mean Distance from Mean coordinates
0.025927370015425652


In [48]:
maps[cities[0]]
#ISTANBUL

In [67]:
maps[cities[3]]
#ANKARA